# F10 Hackathon APIs Example Showcase

In [1]:
import requests
from collections import namedtuple
import json

In [2]:
BANK_API_1="https://f10-sg.cloud.tyk.io/core-bank-api-wgi-1"
AUTH={'Authorization': '5b514bf0fc2fd20001f4afec422132c21c8343ea91a8a2ae5e047b12'}

In [3]:
BANK_API_1="http://localhost:9090"
AUTH={'Authorization': '5b514bf0fc2fd20001f4afec422132c21c8343ea91a8a2ae5e047b12'}

## The Core Banking API 

In [35]:
class MyBanking:
    
    def __init__(self, base_url, token, my_name, my_bank_code):
        self.base_url = base_url
        self.token = token
        self.my_name = my_name
        self.my_bank_code = my_bank_code
        self.accounts = { a.id: a for a in self.list_accounts() }
        
    def list_accounts(self, account_number = None ):
        response = requests.get("{}/accounts".format(self.base_url), headers=self.token)
        return [namedtuple("Account", a.keys())(*a.values()) for a in response.json()
               if a['bank_code'] == self.my_bank_code and (account_number == None or a['account_number'] == account_number)]
        
    def create_account(self, account_number, currency, product_type):
        a = {'id': account_number, 'bank_code': self.my_bank_code, 'account_number': account_number, 'name': self.my_name, 'currency': currency, 'product_type': product_type}
        response = requests.post("{}/accounts".format(self.base_url), headers=self.token, json=a)
        named_tuple = namedtuple("Account", a.keys())(*a.values())
        self.accounts[account_number] = named_tuple
        return named_tuple
    
    def get_balance(self, account_number):
        b = requests.get("{}/accounts/{}/balance".format(self.base_url, account_number), headers=self.token)
        b = b.json()
        return namedtuple("Balance", b.keys())(*b.values())
    
    def auth_payment(self, amount, account, other_party_acct_no, other_party_bank_code, other_party_name, refid, comments):
        tx = {
          "amount": amount,
          "bank_code": other_party_bank_code,
          "comments": comments,
          "credit_account_number": other_party_acct_no,
          "destination_currency": "USD",
          "payee_name": other_party_name,
          "reference_id": refid,
          "source_currency": "USD",
          "transfer_currency": "USD",
          "transfer_type": "INSTANT",
          "value_date": "2018-08-03"
        }
        res = requests.post("{}/accounts/{}/transfer".format(self.base_url, account), json=tx, headers=self.token)
        return res.json()

    def list_transactions(self, account_number):
        return requests.get("{}/accounts/{}/transactions".format(self.base_url, account_number), headers=self.token).json()

## The Singapore National Bank is used to finance our challenge

In [5]:
ACCT_F='000-000-000'
cb_sgnb = MyBanking(BANK_API_1, AUTH, 'SGNB', 'SGNB')
cb_sgnb.create_account(ACCT_F, 'USD', 'FINANCING')

Account(id='000-000-000', bank_code='SGNB', account_number='000-000-000', name='SGNB', currency='USD', product_type='FINANCING')

In [6]:
cb_sgnb.get_balance(ACCT_F)

Balance(id='000-000-000', bank_code='SGNB', account_number='000-000-000', name='SGNB', balance=9000000000, currency='USD', product_type='FINANCING')

You can see, that the magic product type 'FINANCING' created the inital amount that we all live from

## Joe's private core banking accounts

In [7]:
JOE_SAVINGS = '123-234-345'
JOE_CHECK = '123-234-666'

In [25]:
cb_joe = MyBanking(BANK_API_1, AUTH, 'Joe', 'HSBC')

In [9]:
cb_joe.create_account(JOE_SAVINGS, 'USD', 'Saving')

Account(id='123-234-345', bank_code='HSBC', account_number='123-234-345', name='Joe', currency='USD', product_type='Saving')

In [10]:
cb_joe.create_account(JOE_CHECK, 'USD', 'Check')

Account(id='123-234-666', bank_code='HSBC', account_number='123-234-666', name='Joe', currency='USD', product_type='Check')

In [11]:
cb_joe.list_accounts()

[Account(id='123-234-345', bank_code='HSBC', account_number='123-234-345', name='Joe', currency='USD', product_type='Saving'),
 Account(id='123-234-666', bank_code='HSBC', account_number='123-234-666', name='Joe', currency='USD', product_type='Check')]

In [12]:
cb_joe.get_balance(JOE_SAVINGS), cb_joe.get_balance(JOE_CHECK)

(Balance(id='123-234-345', bank_code='HSBC', account_number='123-234-345', name='Joe', balance=0, currency='USD', product_type='Saving'),
 Balance(id='123-234-666', bank_code='HSBC', account_number='123-234-666', name='Joe', balance=0, currency='USD', product_type='Check'))

#### Joe is financed by the Singapore National Bank

In [13]:
cb_sgnb.auth_payment(90000, ACCT_F, JOE_SAVINGS, 'HSBC', 'Joe', '0007601280223', 'initial financing')
cb_sgnb.auth_payment(9000, ACCT_F, JOE_CHECK, 'HSBC', 'Joe', '0007601280224', 'initial financing')

{'comment': 'Fund Transfer was Successfully processed',
 'reference_id': '5b652d49fdf7e90b6c2a304f-5b652d49fdf7e90b6c2a3050',
 'status': 'ACCEPTED'}

In [15]:
cb_joe.get_balance(JOE_SAVINGS), cb_joe.get_balance(JOE_CHECK)

(Balance(id='123-234-345', bank_code='HSBC', account_number='123-234-345', name='Joe', balance=90000, currency='USD', product_type='Saving'),
 Balance(id='123-234-666', bank_code='HSBC', account_number='123-234-666', name='Joe', balance=9000, currency='USD', product_type='Check'))

In [21]:
cb_joe.list_transactions(JOE_SAVINGS)

{'data': [{'account_id': '123-234-345',
   'amount': 90000,
   'booking_date_time': '2018-08-04T06:36:24.985',
   'credit_debit_indicator': 'Credit',
   'currency': 'USD',
   'id': '5b652d48fdf7e90b6c2a304e',
   'transaction_code': 'Fund Transfer',
   'transaction_reference': '000-000-000',
   'value_date_time': '2018-08-04T06:36:24.985'}]}

# Singapore Credit Default Insurance Ltd. 

In [22]:
SCDI_FEES = '111-234-345'
SCDI_CLAIMS = '111-234-666'

In [24]:
cb_scdi = MyBanking(BANK_API_1, AUTH, 'SCDI', 'BJB')

In [31]:
cb_scdi.create_account(SCDI_CLAIMS, 'USD', 'Claims')
cb_scdi.create_account(SCDI_FEES, 'USD', 'Fees')

Account(id='111-234-345', bank_code='BJB', account_number='111-234-345', name='SCDI', currency='USD', product_type='Fees')

In [32]:
cb_sgnb.auth_payment(90000000, ACCT_F, SCDI_CLAIMS, 'BJB', 'SCDI', '0007601280225', 'initial financing')
cb_sgnb.auth_payment(9000, ACCT_F, SCDI_FEES, 'BJB', 'SCDI', '0007601280226', 'initial financing')

{'comment': 'Fund Transfer was Successfully processed',
 'reference_id': '5b6531c9fdf7e90b6c2a3055-5b6531c9fdf7e90b6c2a3056',
 'status': 'ACCEPTED'}

In [33]:
cb_scdi.get_balance(SCDI_CLAIMS), cb_scdi.get_balance(SCDI_FEES)

(Balance(id='111-234-666', bank_code='BJB', account_number='111-234-666', name='SCDI', balance=90000000, currency='USD', product_type='Claims'),
 Balance(id='111-234-345', bank_code='BJB', account_number='111-234-345', name='SCDI', balance=9000, currency='USD', product_type='Fees'))